In [1]:
import keras
import keras.backend as K
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image

from imageio import imread
from sklearn.model_selection import RepeatedKFold
from multiprocessing import Pool
from skimage.filters import sobel_h,sobel_v
from skimage.measure import block_reduce

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing import text, sequence
from keras.losses import binary_crossentropy
from keras.utils import to_categorical
from keras.models import Model, Input, load_model
from keras.layers import BatchNormalization, add, Dropout, Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.metrics import categorical_accuracy
from keras import backend as K
from keras.regularizers import l2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
import sys
import numpy as np
import keras
from keras.utils import Sequence
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import os
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
import keras
import keras.backend as K
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os

from imgaug import augmenters as iaa
from imageio import imread
from sklearn.model_selection import RepeatedKFold
from multiprocessing import Pool
from skimage.filters import sobel_h,sobel_v
from skimage.measure import block_reduce

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing import text, sequence
from keras.losses import binary_crossentropy
from keras.utils import to_categorical
from keras.models import Model, Input, load_model
from keras.layers import BatchNormalization, add,Cropping2D, Dropout, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, concatenate, UpSampling2D
from keras.metrics import categorical_accuracy
from keras import backend as K
from keras.regularizers import l2
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score

Using TensorFlow backend.


# Directory

In [2]:
train_path = "/Users/mohamed/Desktop/Kaggle-Protein/train"

In [3]:
train_labels = pd.read_csv("/Users/mohamed/Desktop/Kaggle-Protein/train.csv")

In [4]:
splitter = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)

In [5]:
partitions = []

for train_idx, test_idx in splitter.split(train_labels.index.values):
    partition = {}
    partition["train"] = train_labels.Id.values[train_idx]
    partition["validation"] = train_labels.Id.values[test_idx]
    partitions.append(partition)
    #print("TRAIN:", train_idx, "TEST:", test_idx)
    #print("TRAIN:", len(train_idx), "TEST:", len(test_idx))

In [6]:
labels_path = "/Users/mohamed/Desktop/Kaggle-Protein/train.csv"

In [7]:
labels = train_labels

# Threshold for prediction: https://www.kaggle.com/iafoss/pretrained-resnet34-with-rgby-0-460-public-lb

In [8]:
lb_prob = [
 0.362397820,0.043841336,0.075268817,0.059322034,0.075268817,
 0.075268817,0.043841336,0.075268817,0.010000000,0.010000000,
 0.010000000,0.043841336,0.043841336,0.014198783,0.043841336,
 0.010000000,0.028806584,0.014198783,0.028806584,0.059322034,
 0.010000000,0.126126126,0.028806584,0.075268817,0.010000000,
 0.222493880,0.028806584,0.010000000]

# Load images

In [9]:
def load_image(basepath, image_id):

    images = np.zeros(shape=(512,512,4))
    images[:,:,0] = Image.open(basepath + "/" + image_id + "_green" + ".png")
    images[:,:,1] = Image.open(basepath + "/" + image_id + "_red" + ".png")
    images[:,:,2] = Image.open(basepath + "/" + image_id + "_blue" + ".png")
    images[:,:,3] = Image.open(basepath + "/" + image_id + "_yellow" + ".png")

    return images

# Transform labels in binary

In [10]:
def get_label(file_ids, train_labels):
    l=train_labels[train_labels.Id.isin(file_ids)]["Target"].values
    l= [[int(i) for i in s.split()] for s in l]
    labels = []
    for i in l:
        L = [0]*28
        for j in i:
            L[j]=1
        labels.append(L)
    return np.array(labels)

In [11]:
for i in get_label(partition["validation"], train_labels).mean(axis=0):
    print(i)

0.4110074026392018
0.04184100418410042
0.115545542323785
0.05133569359510782
0.06002574831026714
0.08255551979401352
0.03427743804312842
0.09349855165754747
0.002574831026713872
0.001126488574187319
0.000643707756678468
0.035082072738976504
0.023495333118764082
0.017219182491149017
0.03057611844222723
0.00048278081750885096
0.0160926939169617
0.006598004505954297
0.03105889925973608
0.051496620534277435
0.007885420019311232
0.123109108464757
0.0246218216929514
0.09221113614419053
0.01013839716768587
0.2597360798197618
0.010621177985194722
0.000160926939169617


# Features augmentation 

In [12]:
def grad(image):
    grad_x0=sobel_h(image[:,:,0])
    grad_y0=sobel_v(image[:,:,0])
    grad0=np.sqrt(grad_x0*grad_x0+grad_y0*grad_y0).T
    
    grad_x1=sobel_h(image[:,:,1])
    grad_y1=sobel_v(image[:,:,1])
    grad1=np.sqrt(grad_x1*grad_x1+grad_y1*grad_y1).T
    
    grad_x2=sobel_h(image[:,:,2])
    grad_y2=sobel_v(image[:,:,2])
    grad2=np.sqrt(grad_x2*grad_x2+grad_y2*grad_y2).T
    
    grad_x3=sobel_h(image[:,:,3])
    grad_y3=sobel_v(image[:,:,3])
    grad3=np.sqrt(grad_x3*grad_x3+grad_y3*grad_y3).T
    
    return np.array([grad0,grad1,grad2, grad3]).T

def grad_threshold(image, eps):
    
    return (grad(image) > eps)*255

In [13]:
def dirrectional_grad(image,theta):
    
    grad_x0 = np.cos(theta)*sobel_h(image[:,:,0]) + np.sin(theta)*sobel_v(image[:,:,0])
    grad_x1 = np.cos(theta)*sobel_h(image[:,:,1]) + np.sin(theta)*sobel_v(image[:,:,1])
    grad_x2 = np.cos(theta)*sobel_h(image[:,:,2]) + np.sin(theta)*sobel_v(image[:,:,2])
    grad_x3 = np.cos(theta)*sobel_h(image[:,:,3]) + np.sin(theta)*sobel_v(image[:,:,3])
    
    grad0= np.maximum(grad_x0,0).T
    grad1= np.maximum(grad_x1,0).T
    grad2= np.maximum(grad_x2,0).T
    grad3= np.maximum(grad_x3,0).T
    image= np.array([grad0,grad1,grad2, grad3]).T
    return image

# Find the shape according to preprocessing choices

In [14]:
def shape(grad, dir_grad, grad_threshold, nb_rot, nb_threshold, block_size):
    if block_size != 0: 
        h = 512//block_size
        h_rest = (512%block_size)>0
    else:
        h, h_rest= 512, 0
        
    return (h+h_rest, h+h_rest, 4 + 4*grad + dir_grad*4*len(np.arange(0, 360, 360//nb_rot)) + grad_threshold*4*len(np.arange(0,128,128//nb_threshold)))

# Define the metrics

In [15]:
def fbeta_score_macro(y_true, y_pred, beta=1, threshold=lb_prob):

    y_true = K.cast(y_true, 'float')
    y_pred = K.cast(K.greater(K.cast(y_pred, 'float'), threshold), 'float')

    tp = K.sum(y_true * y_pred, axis=0)
    fp = K.sum((1 - y_true) * y_pred, axis=0)
    fn = K.sum(y_true * (1 - y_pred), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = (1 + beta ** 2) * p * r / ((beta ** 2) * p + r + K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)

    return K.mean(f1)

# Generate data

In [16]:
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, list_IDs, labels_path, shuffle, batch_size, basepath, grad, dir_grad, grad_threshold, nb_threshold, nb_rot, reduce, block_size):

        self.labels = pd.read_csv(labels_path)
        self.list_IDs = list_IDs
        self.batch_size = batch_size
        self.shape_features = shape(grad, dir_grad, grad_threshold, nb_rot, nb_threshold, block_size)
        self.batch_shape = (self.batch_size, self.shape_features[0], self.shape_features[1], self.shape_features[2]) 
        self.shuffle = shuffle
        self.on_epoch_end()
        self.basepath = basepath
        self.grad = grad
        self.dir_grad = dir_grad
        self.grad_threshold = grad_threshold
        self.nb_threshold = nb_threshold 
        self.nb_rot = nb_rot
        self.reduce = reduce
        self.block_size = block_size
    
    def Reduce(self, images):
        block=(self.block_size,self.block_size,1)
        return block_reduce(images,block,np.mean)
        
    def features_aumentation(self, image):    

        grad_=self.grad
        dir_grad_=self.dir_grad
        grad_threshold_=self.grad_threshold

        a=image

        if grad_:
            a=np.append(a,grad(image), axis=2)

        if dir_grad_:
            rot=np.arange(0, 360, 360//self.nb_rot)
            for i in rot:
                a=np.append(a,dirrectional_grad(image,i), axis=2)

        if grad_threshold_:
            eps=np.arange(0,128,128//self.nb_threshold)
            for e in eps:      
                a=np.append(a, grad_threshold(image,e), axis=2)

        return a
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def get_loader(self,list_id):

        filenames = []
        idx = 0
        images = np.zeros(self.batch_shape)

        for image_id in list_id:
            if self.reduce:
                images[idx,:,:,:] = self.Reduce(self.features_aumentation(load_image(self.basepath, image_id)))
            else:
                images[idx,:,:,:] = self.features_aumentation(load_image(self.basepath, image_id))
            
            filenames.append(image_id)
            idx += 1
            if idx == self.batch_shape[0]:
                images = np.array(images)

                return filenames, get_label(filenames,self.labels), images
                
                filenames = []
                images = np.zeros(self.batch_shape)
                idx = 0
        
        if idx > 0:

            images = np.array(images)
            return filenames, get_label(filenames,self.labels), images
   
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        Ids, y, X = self.get_loader(list_IDs_temp)
        return X, y

In [17]:
class PredictGenerator:
    
    def __init__(self, predict_Ids, predict_path):
        self.basepath = predict_path
        self.identifiers = predict_Ids        
    
    def predict(self, model):
        y = np.empty(shape=(len(self.identifiers), 28))
        for n in range(len(self.identifiers)):
            image = load_image(self.basepath, self.identifiers[n])
            image = self.features_aumentation(self, image)
            image = image.reshape((1, *image.shape))
            #print(image.shape)
            y[n] = model.predict(image)
        return y

# Split the data set

In [18]:
splitter = RepeatedKFold(n_splits=3, n_repeats=1, random_state=0)

In [19]:
partitions = []

for train_idx, test_idx in splitter.split(train_labels.index.values):
    partition = {}
    partition["train"] = train_labels.Id.values[train_idx]
    partition["validation"] = train_labels.Id.values[test_idx]
    partitions.append(partition)
    #print("TRAIN:", train_idx, "TEST:", test_idx)
    #print("TRAIN:", len(train_idx), "TEST:", len(test_idx))

In [20]:
len(partition["validation"])

10357

# Define generator of data with preprocessing: validation and training

In [21]:
train_generator = DataGenerator(list_IDs= partition['train'],
                                   batch_size = 64,
                                   shuffle = True,
                                  labels_path= labels_path, 
                                   basepath= train_path, 
                                  grad = True, 
                                   dir_grad = True, 
                                   grad_threshold = True,
                                 nb_rot= 4, 
                                   nb_threshold=6, 
                                   reduce=True, 
                                   block_size=3)

In [22]:
validation_generator = DataGenerator(partition['validation'],
                                   batch_size = 64,
                                   shuffle = True,
                                  labels_path= labels_path, 
                                   basepath= train_path, 
                                  grad = True, 
                                   dir_grad = True, 
                                   grad_threshold = True,
                                 nb_rot= 4, 
                                   nb_threshold=6, 
                                   reduce=True, 
                                   block_size=3)

In [23]:
predict_generator = PredictGenerator(partition['validation'], train_path)

In [24]:
def get_crop_shape(target, refer):
    # width, the 3rd dimension
    cw = (target.get_shape()[2] - refer.get_shape()[2]).value
    assert (cw >= 0)
    if cw % 2 != 0:
        cw1, cw2 = int(cw/2), int(cw/2) + 1
    else:
        cw1, cw2 = int(cw/2), int(cw/2)
    # height, the 2nd dimension
    ch = (target.get_shape()[1] - refer.get_shape()[1]).value
    assert (ch >= 0)
    if ch % 2 != 0:
        ch1, ch2 = int(ch/2), int(ch/2) + 1
    else:
        ch1, ch2 = int(ch/2), int(ch/2)

    return (ch1, ch2), (cw1, cw2)

In [25]:
class BaseLineModel:
    
    def __init__(self, n_epochs):

        self.input_shape = train_generator.shape_features
        self.my_metrics = ['accuracy', fbeta_score_macro]
        self.n_epochs = n_epochs
        #print(self.input_shape)
    
    def build_model(self):
        
        #Useless model
        '''self.model = Sequential()'''
        '''self.model.add(Conv2D(16, kernel_size=(3, 3), activation='relu',input_shape=self.input_shape))
        self.model.add(Conv2D(32, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        self.model.add(Flatten())
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(28, activation='sigmoid'))'''
        
        #Useless model 2
        input = Input(shape = self.input_shape )
        
        x= input
        
        A = Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu', kernel_regularizer=l2(0.001))(x)
        A = MaxPooling2D(pool_size=2, strides=1, padding="valid")(A)
        A= BatchNormalization()(A)

        B = Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu', kernel_regularizer=l2(0.001))(A)
        B = MaxPooling2D(pool_size=2, strides=1, padding="valid")(B)
        B= BatchNormalization()(B)

        C = Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu', kernel_regularizer=l2(0.001))(B)
        C = MaxPooling2D(pool_size=2, strides=1, padding="valid")(C)
        C= BatchNormalization()(C)

        a = Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu', kernel_regularizer=l2(0.001))(x)
        a = MaxPooling2D(pool_size=2, strides=1, padding="valid")(a)
        a= BatchNormalization()(a)

        b = Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu', kernel_regularizer=l2(0.001))(x)
        b = MaxPooling2D(pool_size=2, strides=1, padding="valid")(b)
        b= BatchNormalization()(b)

        c = Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu', kernel_regularizer=l2(0.001))(x)
        c = MaxPooling2D(pool_size=2, strides=1, padding="valid")(c)
        c= BatchNormalization()(c)

        x = add([a,b,c])

        C= Flatten()(C)
        C=Dropout(0.3)(C)

        x= Flatten()(x)
        x=Dropout(0.3)(x)

        x = Dense((256),activation='relu', kernel_regularizer=l2(0.001))(x)
        C = Dense((256),activation='relu', kernel_regularizer=l2(0.001))(C)

        x = add([x,C])

        x=Dropout (0.3)(x)
        x=Dense((512),activation='relu', kernel_regularizer=l2(0.001))(x)

        output = Dense(28, activation = "sigmoid")(x)

        self.model = Model(inputs=[input], outputs=[output])
        
        #Spoon model
        '''concat_axis = 3
        input = Input(self.input_shape)

        conv1 = Conv2D(32, (3, 3), padding="same", name="conv1_1", kernel_initializer='uniform',activation="relu", data_format="channels_last")(input)
        conv1 = Conv2D(32, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)
        pool1= BatchNormalization()(pool1)

        conv2 = Conv2D(64, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(pool1)
        conv2 = Conv2D(64, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)
        pool2= BatchNormalization()(pool2)

        conv3 = Conv2D(128, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(pool2)
        conv3 = Conv2D(128, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv3)
        pool3= BatchNormalization()(pool3)

        conv4 = Conv2D(256, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(pool3)
        conv4 = Conv2D(256, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv4)
        pool4= BatchNormalization()(pool4)

        conv5 = Conv2D(512, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(pool4)
        conv5 = Conv2D(512, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(conv5)

        up_conv5 = UpSampling2D(size=(2, 2),data_format="channels_last")(conv5)
        ch, cw = get_crop_shape(conv4, up_conv5)
        crop_conv4 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv4)
        up6   = concatenate([up_conv5, crop_conv4], axis=concat_axis)
        conv6 = Conv2D(256, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(up6)
        conv6 = Conv2D(256, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(conv6)

        up_conv6 = UpSampling2D(size=(2, 2),data_format="channels_last")(conv6)
        ch, cw = get_crop_shape(conv3, up_conv6)
        crop_conv3 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv3)
        up7   = concatenate([up_conv6, crop_conv3], axis=concat_axis)
        conv7 = Conv2D(128, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(up7)
        conv7 = Conv2D(128, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(conv7)

        up_conv7 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv7)
        ch, cw = get_crop_shape(conv2, up_conv7)
        crop_conv2 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv2)
        up8   = concatenate([up_conv7, crop_conv2], axis=concat_axis)
        conv8 = Conv2D(64, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(up8)
        conv8 = Conv2D(64, (3, 3), padding="same", activation="relu", kernel_initializer='uniform',data_format="channels_last")(conv8)

        up_conv8 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv8)
        ch, cw = get_crop_shape(conv1, up_conv8)
        crop_conv1 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv1)
        up9   = concatenate([up_conv8, crop_conv1], axis=concat_axis)
        conv9 = Conv2D(32, (3, 3), padding="same", activation="relu", kernel_initializer='uniform', data_format="channels_last")(up9)
        conv9 = Conv2D(32, (3, 3), padding="same", activation="relu", kernel_initializer='uniform', data_format="channels_last")(conv9)

        x= conv9

        A = Conv2D(32, kernel_size=3, strides=2, padding='valid', activation='relu', kernel_initializer='uniform',kernel_regularizer=l2(0.001))(x)
        A = MaxPooling2D(pool_size=2, strides=1, padding="valid")(A)
        A= BatchNormalization()(A)

        B = Conv2D(32, kernel_size=3, strides=2, padding='valid', activation='relu', kernel_initializer='uniform',kernel_regularizer=l2(0.001))(A)
        B = MaxPooling2D(pool_size=2, strides=1, padding="valid")(B)
        B= BatchNormalization()(B)

        C = Conv2D(32, kernel_size=3, strides=1, padding='valid', activation='relu', kernel_initializer='uniform',kernel_regularizer=l2(0.001))(B)
        C = MaxPooling2D(pool_size=2, strides=1, padding="valid")(C)
        C= BatchNormalization()(C)

        D = Conv2D(32, kernel_size=3, strides=1, padding='valid', activation='relu', kernel_initializer='uniform',kernel_regularizer=l2(0.001))(C)
        D = MaxPooling2D(pool_size=2, strides=1, padding="valid")(D)
        D = BatchNormalization()(D)

        E = Conv2D(32, kernel_size=3, strides=1, padding='valid', activation='relu', kernel_initializer='uniform',kernel_regularizer=l2(0.001))(D)
        E = MaxPooling2D(pool_size=2, strides=1, padding="valid")(E)
        E = BatchNormalization()(E)

        F = Conv2D(32, kernel_size=3, strides=1, padding='valid', activation='relu', kernel_initializer='uniform',kernel_regularizer=l2(0.001))(E)
        F = MaxPooling2D(pool_size=2, strides=1, padding="valid")(F)
        F = BatchNormalization()(F)

        G = Conv2D(32, kernel_size=3, strides=1, padding='valid', activation='relu', kernel_initializer='uniform',kernel_regularizer=l2(0.001))(F)
        G = MaxPooling2D(pool_size=2, strides=1, padding="valid")(G)
        G = BatchNormalization()(G)

        H = Conv2D(32, kernel_size=3, strides=1, padding='valid', activation='relu', kernel_initializer='uniform',kernel_regularizer=l2(0.001))(G)
        H = MaxPooling2D(pool_size=2, strides=1, padding="valid")(H)
        H = BatchNormalization()(H)

        H= Flatten()(H)
        H= Dropout(0.3)(H)

        x = Dense((128),activation='relu', kernel_regularizer=l2(0.001), kernel_initializer='uniform')(H)

        x=Dropout (0.3)(x)
        x=Dense((256),activation='relu', kernel_regularizer=l2(0.001), kernel_initializer='uniform')(x)

        output = Dense(28, activation = "sigmoid", kernel_initializer='uniform')(x)

        self.model = Model(inputs=[input], outputs=[output])'''

        return self.model
        
    def compile_model(self):
        self.model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Nadam(),
              metrics=self.my_metrics)
    
    def set_generators(self, train_generator, validation_generator):
        self.training_generator = train_generator
        self.validation_generator = validation_generator
    
    def learn(self):
        return self.model.fit_generator(generator=self.training_generator,
                    validation_data=self.validation_generator,
                    epochs=self.n_epochs, 
                    use_multiprocessing=True,
                    workers=8)
    
    def score(self):
        return self.model.evaluate_generator(generator=self.validation_generator,
                                      use_multiprocessing=True, 
                                      workers=8)
    
    def predict(self, predict_generator):
        y = predict_generator.predict(self.model)
        return y
    
    def save(self, modeloutputpath):
        self.model.save(modeloutputpath)
    
    def load(self, modelinputpath):
        self.model = load_model(modelinputpath)

# BaseLineModel for 10 Epochs

In [26]:
model = BaseLineModel(10)
model.build_model()
model.compile_model()

In [27]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

AttributeError: 'BaseLineModel' object has no attribute 'layers'

In [ ]:
'''model.set_generators(train_generator, validation_generator)
history = model.learn()
model.save("baseline_model.hdf5")'''

https://github.com/yihui-he/u-net/blob/master/train.py

In [ ]:
'''def Unet():
    inputs = Input(self.input_shape)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)
    
    up6 = merge([Convolution2D(256, 2, 2,activation='relu', border_mode='same')(UpSampling2D(size=(2, 2))(conv5)), conv4], mode='concat', concat_axis=1)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([Convolution2D(128, 2, 2,activation='relu', border_mode='same')(UpSampling2D(size=(2, 2))(conv6)), conv3], mode='concat', concat_axis=1)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([Convolution2D(64, 2, 2,activation='relu', border_mode='same')(UpSampling2D(size=(2, 2))(conv7)), conv2], mode='concat', concat_axis=1)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([Convolution2D(32, 2, 2,activation='relu', border_mode='same')(UpSampling2D(size=(2, 2))(conv8)), conv1], mode='concat', concat_axis=1)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model'''